# Scope, Closure, Decorator

In [1]:
# built-in, module, global, local, non-local scopes

In [3]:
a = 10

def func():
    print(a)
    a = 100

func()

UnboundLocalError: local variable 'a' referenced before assignment

In [4]:
a = 10

def func():
    global a
    a = 100

func()
print(a)

100


In [6]:
def outer():
    x = "Hello"
    
    def inner():
        nonlocal x
        x = "Python"
    
    print("Outer(before)", x)
    inner()
    print("Inner", x)
    print("Outer(after)", x)
outer()

Outer(before) Hello
Inner Python
Outer(after) Python


## Closure 

In [10]:
# inner function + local variables
# Closure is a function with free variables

def outer():
    # ----- Closure start -----
    x = "python"
    
    def inner():
        print(x)
    # ----- Closure end -----
    
    return inner

cl = outer()
cl()

python


In [11]:
# Closure is a function with free variables
cl.__code__.co_freevars

('x',)

In [21]:
# closures with same freevar
def outer():
    count = 0
    
    def inner1():
        nonlocal count
        count += 1
        return count
    def inner2():
        nonlocal count
        count += 1
        return count
    
    return inner1, inner2

cl1, cl2 = outer()

In [16]:
cl1.__code__.co_freevars, cl2.__code__.co_freevars

(('count',), ('count',))

In [17]:
cl2.__code__.co_freevars

('count',)

In [18]:
cl1()

1

In [19]:
cl1()

2

In [20]:
cl2()

3

In [52]:
from functools import wraps
# counting function calls

def add(a, b):
    return a+b

def mult(a, b):
    return a*b

func_counts = dict()


def counter(fn, func_counts):
    count = 0
    
    @wraps(fn) # for storing docstring, annotations and others.
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        func_counts[fn.__name__] = count
        return fn(*args, **kwargs)
    return inner

fac = counter(fac, func_counts)

In [53]:
add = counter(add, func_counts)
mult = counter(mult, func_counts)

In [54]:
add(4,5)
mult(2,3)
add(2,3)
add(1,2)

func_counts

{'add': 3, 'mult': 1}

## Decorator

In [58]:
def counter_factory(func_counts):
    def counter(fn):
        count = 0

        @wraps(fn) # for storing docstring, annotations and others.
        def inner(*args, **kwargs):
            nonlocal count
            count += 1
            func_counts[fn.__name__] = count
            return fn(*args, **kwargs)
        return inner
    return counter

In [59]:
counter = counter_factory(func_counts)

In [61]:
@counter_factory(func_counts)
def pow(a, b):
    return a**b

In [63]:
pow(3,4)
add(2,3)
mult(1,2)
pow(6,7)

func_counts

{'add': 4, 'mult': 2, 'pow': 3}